In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')


In [2]:
import time
import os

from datetime import date
from datetime import timedelta
from datetime import datetime

import pickle

import json

import pandas as pd
import numpy as np
import random 

from handle_data.read_data import load_candles

from backtesting import Strategy
from backtesting import Backtest

import pandas_ta as ta


Loading BokehJS ...

In [3]:
history_days = 10 * 30

sim_symbol = "BTC/USDT"
asset_factor = 1_000_000

#sim_symbol = "ETHBUSD"
#asset_factor = 1_000

#sim_symbol = "MATICBTC"
#asset_factor = 1

start_date = (date.today() - timedelta(days=history_days)).strftime('%Y-%m-%d')
end_date=None

start_date = "2019-11-01"
#end_date="2021-07-22"
#end_date="2021-04-14"

print("start_date:",start_date)
print("end_date:",end_date)


data = load_candles('Binance', sim_symbol, timeframe='1d', factor=asset_factor, start_date=start_date, end_date=end_date, )

start_date: 2019-11-01
end_date: None
reading ../data/BTC-USDT/1d/BTC-USDT_1d_Binance.pickle


FileNotFoundError: [Errno 2] No such file or directory: '../data/BTC-USDT/1d/BTC-USDT_1d_Binance.pickle'

In [ ]:
class DCA_Conditional_Buy_LR_with_TrailingStop(Strategy):  
    # DCA + Moving Stop Loss
    
    band_lenght = 13
    band_mult = 1
    mode_band_dev = 1
    
    intial_sl_factor = 0.0
    sl_activation_margin = 1.003
    
    lower_band_prct = 0.95
    
    sleep_by = 0

    reinvest = 1
    stop_loose_changes = 0
    i_cash = 1_000
    
    buy_all_days = 0
    dca_budget = 100
    lr_buy_longitude = 3
    
    
    def lb_close(self,data): #lower_band_stdev_formula
        dema = ta.dema(close = data.Close.s, length=self.band_lenght).to_numpy()
        stdev = ta.stdev(close = data.Close.s, length=self.band_lenght).to_numpy()
        lower_band = dema - ( self.band_mult * stdev )
        return lower_band
    
    def lb_low(self,data): # lower_band_stdev_formula_on_low
        dema = ta.dema(close = data.Low.s, length=self.band_lenght).to_numpy()
        stdev = ta.stdev(close = data.Low.s, length=self.band_lenght).to_numpy()
        lower_band = dema - ( self.band_mult * stdev )
        return lower_band

    
    def LR(self,data):
        # https://greyhoundanalytics.com/blog/custom-indicators-in-backtestingpy/
        linreg = ta.linreg(close = data.Close.s, length=self.lr_buy_longitude, angle=True )
        return linreg.to_numpy()
    
    
    
    
    def calculate_size(self):
        if self.reinvest==0:
            if self.equity < self.i_cash:
                cash = self.equity
            else:
                cash = self.i_cash
            size = int((cash / self.data.Close[-1]) * 0.99 )
            #print(f"buy with limit {size} = {self.i_cash} / {self.data.Close[-1]}")
        elif self.reinvest==1:
            size = int( (self.equity / self.data.Close[-1]) * 0.99 )
            #print(f"buy no limit {size} = {self.equity} / {self.data.Close[-1]} ")
        elif self.reinvest==2:
            if self.equity < self.dca_budget:
                cash = self.equity
            else:
                cash = self.dca_budget
            size = int( ( cash / self.data.Close[-1]) * 0.99 )

        return size
    
    def should_sleep(self):
        if len(self.closed_trades)>0:
            last_trade_exit_time=self.closed_trades[-1].exit_time
            #print('last_trade_exit_time',last_trade_exit_time)
            
            current_time_stamp = self.data.index[-1]
            #print('current_time_stamp',current_time_stamp)
            
            time_stamp_difference = (current_time_stamp - last_trade_exit_time).total_seconds() / 60
            #print('time_stamp_difference',time_stamp_difference)
            
            sleep = (time_stamp_difference < self.sleep_by)
            #print('sleep',sleep)
            
            return sleep
        else:
            return False # if no trades no sleeps
    
    def init(self):
        self.day_of_week = self.I(lambda x: x,self.data.Close.s.index.dayofweek,plot = False,)
        self.lr_buy = self.I(self.LR, self.data,plot = False)
        self.lower_band = self.I(self.lb_close, self.data)
        self.lb_low = self.I(self.lb_low, self.data)

        self.current_stop_loss = 0 
        self.i_cash = self.equity
        self.dca_bought = False
        
    def do_buy(self,sl):
        self.current_stop_loss = sl * self.intial_sl_factor
        size=self.calculate_size()
        if size>0:
            self.buy(size=size,sl=self.current_stop_loss) 
        
    def next(self):

        new_stop = self.lower_band[-1]
        new_stop_price = self.lb_low[-1]
        
        if self.reinvest in [0,1]:
            if not self.position:
                if not self.should_sleep(): 
                    self.do_buy(new_stop)
                    self.sleep_by = random.random() * 7 * 24 * 60 
        else:
            if self.day_of_week[-1] == 1 or self.buy_all_days: 
                if self.dca_bought == False and self.lr_buy[-1]>0:
                    self.do_buy(new_stop)
                    self.dca_bought == True
            else:
                self.dca_bought == False
        
         
        if new_stop > self.current_stop_loss:
            self.current_stop_loss = new_stop
            for trade in self.trades:
                if new_stop_price > trade.entry_price * self.sl_activation_margin:
                    old_stop = trade.sl
                    if self.data.Close[-1] > new_stop:
                        trade.sl = new_stop
                        #print(f"I'm recreating the stop loss {old_stop}, new: {new_stop}. Entry price {self.trades[0].entry_price}")
            else:
                #print(f"ERROR, I'm trying to create the stop loss {old_stop}, new: {new_stop}. Entry price {self.trades[0].entry_price}")
                pass

In [ ]:
start_time = time.time()

In [ ]:
def opt_asset_factor(stats):
    trades = stats._trades
    if len(trades)>0:
        last_trade = trades["ExitBar"].max()
        final_holding = trades[trades["ExitBar"] == last_trade]["Size"].sum()
        final_open = trades[trades["ExitBar"] == last_trade]["Size"].count()
        return final_holding / ( trades['Size'][0] * final_open )
    else:
        return 1

In [ ]:
fb = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]

In [ ]:
# %%script false --no-raise-error

bt = Backtest(data, DCA_Conditional_Buy_LR_with_TrailingStop, cash=1600, commission=.001)
optimize = True

if optimize:
     stats = bt.optimize(maximize="Return [%]",
                        #band_lenght=[*range(3, 11, 1)],
                        band_lenght=[3,9,10,11,12,13,21,34],
                        #band_mult=[*range(1, 3, 1)],
                        band_mult=[1,2],
                        mode_band_dev = [1],
                        intial_sl_factor=[0], #,0.7,0.8,0.9],
                        #sl_activation_margin=[1.003, 1.004, 1.005, 1.006, 1.007, 1.008, 1.009, 1.01, 1.012 ],
                        sl_activation_margin=[1.003],
                        #maximize=opt_asset_factor,
                        reinvest = [2],
                        dca_budget = [100],
                        buy_all_days=1,
                        lr_buy_longitude=[*range(2, 7, 1)],

    )
else:
    # daily 

    #stats = bt.run(band_lenght=21,band_mult=2,mode_band_dev=1,intial_sl_factor=0.9,sl_activation_margin=1.003,reinvest=2,dca_budget=49,buy_all_days=1,lr_buy_longitude=5)
    stats = bt.run(band_lenght=13,band_mult=1,mode_band_dev=1,
                   intial_sl_factor=0.0,sl_activation_margin=1.003,reinvest=2,
                   dca_budget=49,buy_all_days=1,lr_buy_longitude=5)


stats

In [ ]:
stats._strategy

In [ ]:
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time) )

In [ ]:
bt.plot(filename=datetime.now().strftime('%Y-%m-%d_%H%M')+'.html')

In [ ]:
bt.plot()

In [ ]:
stats._trades


In [ ]:
equity_curve = stats._equity_curve
equity_curve

In [ ]:
opt_asset_factor(stats)

In [ ]:
trades = stats._trades
last_trade = trades["ExitBar"].max()
final_holding = trades[trades["ExitBar"] == last_trade]["Size"].sum()
final_open = trades[trades["ExitBar"] == last_trade]["Size"].count()
final_holding

In [ ]:
final_open

In [ ]:
last_trade

In [ ]:
#BTCBUSD = load_candles(sim_symbol, start_date="2021-01-01", end_date=end_date, factor=asset_factor, resample='1d')
#BTCBUSD